In [9]:
import pandas as pd

In [10]:
re_dust_df = pd.read_csv('../data/seoul_re_dust.csv', index_col=0)
living_df = pd.read_csv('../data/living_df.csv', index_col=0)
fine_dust_df = pd.read_csv('../data/fine_dust_df.csv', index_col=0)
bus_stop_df = pd.read_csv('../data/bus_stop.csv', encoding='cp949')
construct_df = pd.read_csv('../data/contsruct_info.csv')
map_df = pd.read_csv('../data_preprocess/행정동코드_매핑정보_20200325.csv')

In [11]:
re_dust_df['date_cd'] = re_dust_df['date'].apply(lambda x: "".join(x.split('-')))

In [12]:
main_df = re_dust_df.groupby(['date_cd', 'gu'])[['temperature', 'humidity', 'avg_re_dust']].mean().reset_index()

In [13]:
fine_dust_df['MSRDT_DE'] = fine_dust_df['MSRDT_DE'].astype(str)

In [14]:
merged_df = pd.merge(main_df, 
                     fine_dust_df, 
                     left_on=["date_cd", "gu"], 
                     right_on=["MSRDT_DE", "MSRSTE_NM"])

In [15]:
living_df = pd.merge(living_df, map_df, left_on='ADSTRD_CODE_SE', right_on='RESD_CD')

In [16]:
living_df['STDR_DE_ID'] = living_df['STDR_DE_ID'].astype(str)

In [17]:
merged_df2 = pd.merge(
    merged_df, 
    living_df, 
    left_on=["date_cd", "gu"],
    right_on=["STDR_DE_ID", "RESC_CT_NM"]
)

In [18]:
bus_stop_df.columns = ['gu', 'stop_cnt']

In [19]:
merged_df3 = pd.merge(merged_df2, 
                    bus_stop_df, 
                    left_on=["gu"],
                    right_on=["gu"])

In [20]:
from datetime import datetime 

merged_df3['date'] = merged_df3['date_cd'].apply(lambda x: datetime.strptime(x, '%Y%m%d').strftime('%Y-%m-%d'))

In [21]:
construct_cnt_df = pd.DataFrame()

for idx, dt in enumerate(merged_df3['date'].unique()):
    temp = construct_df[(construct_df['cons_start_dt'] <= dt)
                        & (construct_df['cons_end_dt'] > dt)]
    out = pd.DataFrame(temp.groupby('gu')['cons_start_dt'].count()).reset_index()
    out['date'] = dt
    out.columns = ['gu', 'cons_cnt', 'date']
    if idx == 0:
        construct_cnt_df = out
    else:
        construct_cnt_df = pd.concat([construct_cnt_df, out], axis=0)

In [22]:
df = pd.merge(merged_df3,
              construct_cnt_df,
              how='left',
              on=['date', 'gu'],)

In [23]:
columns = [
    'date_cd', 'gu', 'temperature', 'humidity', 'avg_re_dust', 
    'PM10', 'PM25', 'O3', 'NO2', 'CO', 'SO2', 'TOT_LVPOP_CO',
    'stop_cnt', 'cons_cnt'
]

In [24]:
df = df[columns]

In [25]:
column_names = [
    'date_cd', 'gu', 'temp', 'hum', 'redust', 'pm10', 'pm25',
    'o3', 'no2' ,'co', 'so2', 'lvpop', 'stops', 'consts'
]

In [26]:
df.columns = column_names

In [27]:
df.to_csv('../data/main_df.csv')